In [1]:
import os


In [2]:
!pwd

/home/reby/Desktop/personal_Projects/car_price_prediction/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/home/reby/Desktop/personal_Projects/car_price_prediction


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    target_column : str


In [6]:
from src.carPricePrediction.constants import *
from src.carPricePrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            target_column=schema.name
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.carPricePrediction import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        print(train_data.info())
        print(test_data.shape)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        rf_rgrs = RandomForestRegressor(n_estimators=self.config.n_estimators)
        rf_rgrs.fit(train_x, train_y)

        joblib.dump(rf_rgrs, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-07 19:42:09,023: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-07 19:42:09,025: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-07 19:42:09,029: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-07 19:42:09,031: INFO: common: created directory at: artifacts]
[2024-07-07 19:42:09,032: INFO: common: created directory at: artifacts/model_trainer]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            813 non-null    int64  
 1   0                813 non-null    float64
 2   1                813 non-null    float64
 3   2                813 non-null    float64
 4   3                813 non-null    float64
 5   4                813 non-null    float64
 6   5                813 non-null    float64
 7   6                813 non-null    float64
 8   7          

/home/reby/Desktop/personal_Projects/car_price_prediction/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
